In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ecephys.sglx_utils import load_timeseries
from ecephys.signal.csd import get_kcsd
from ecephys.utils import find_nearest
import ecephys_analyses as ea

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from kcsd import KCSD1D
from scipy.signal import find_peaks
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

In [4]:
def get_csd(sig, params):
    k = get_kcsd(
        sig.values,
        intersite_distance=params["intersite_distance"],
        gdx=params["gdx"],
        lambd=params["lambd"],
        R_init=params["R"],
        do_lcurve=False,
    )
    est_csd = k.values('CSD')
    
    return xr.DataArray(
        est_csd,
        dims=("channel", "time"),
        coords={"channel": sig.channel.values, "time": sig.time.values},
        attrs={'units': "nA/mm", 'fs': sig.fs}
    )

In [5]:
def plot_epoched_profile(da, figsize=(36, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(da, xticklabels=da.time.values.round(), yticklabels=da.channel.values, cbar=False)
    ax.set(xticks=ax.get_xticks()[::2], yticks=ax.get_yticks()[::4], xlabel='Epoch center time (s)', ylabel='Channel')

In [6]:
def plot_profile(da, figsize=(36, 5), ylabel=None, negative_peaks=False):
    fig, ax = plt.subplots(figsize=figsize)
    channel_indices = np.arange(len(da.channel))
    sns.barplot(x=channel_indices, y=da, color='steelblue', ax=ax)
    ax.set(xlabel="Channel", ylabel=ylabel)
    ax.set_xticks(channel_indices);
    ax.set_xticklabels(da.channel.values, rotation=90);
    
    if negative_peaks:
        peaks, _ = find_peaks(-da.values, prominence=1000, distance=10)
    else:
        peaks, _ = find_peaks(da.values, prominence=1000, distance=10)
    sns.scatterplot(x=peaks, y=da.values[peaks], marker='x', color='red', ax=ax)

In [7]:
def write_file_report(csd, epoch_length, pdf_path):
    
    if csd.time.values.max() < epoch_length:
        return
    
    epoched_csd_nadirs = csd.coarsen(time=int(csd.fs *epoch_length), boundary='trim', coord_func={"time": "min"}).min()
    epoched_csd_variance = csd.coarsen(time=int(csd.fs *epoch_length), boundary='trim', coord_func={"time": "min"}).var()
    
    pdf_path = pdf_path.parent / (pdf_path.stem + f'-{epoch_length}s' + pdf_path.suffix)
    with PdfPages(pdf_path) as pdf:
        plot_epoched_profile(epoched_csd_variance)
        plt.title(f'CSD variance, {epoch_length}s epochs')
        pdf.savefig()
        plt.close()
        
        plot_epoched_profile(-epoched_csd_nadirs)
        plt.title(f'CSD nadirs, {epoch_length}s epochs')
        pdf.savefig()
        plt.close()

        for epoch in range(epoched_csd_nadirs.shape[1]):
            plot_profile(epoched_csd_variance[:, epoch], ylabel="CSD variance")
            plt.title(f'CSD variance, epoch: {epoch}, time: {epoched_csd_nadirs.time.values[epoch]}s')
            pdf.savefig()
            plt.close()
            
            plot_profile(epoched_csd_nadirs[:, epoch], ylabel="Deepest CSD sink (mA/mm)", negative_peaks=True)
            plt.title(f'CSD nadirs, epoch: {epoch}, time: {epoched_csd_nadirs.time.values[epoch]}s')
            pdf.savefig()
            plt.close()

        d = pdf.infodict()
        d['Title'] = f'{pdf_path.stem}'
        d['Author'] = 'Graham Findlay'
        d['Subject'] = 'CSD nadir profiles for tracking CA1 drift'
        d['CreationDate'] = datetime.now()

In [8]:
def write_file_reports(bin_path, pdf_path, chans, csd_params):
    sig = load_timeseries(bin_path, chans)
    csd = get_csd(sig, csd_params)
    
    write_file_report(csd, 600, pdf_path)
    write_file_report(csd, 300, pdf_path)
    write_file_report(csd, 60, pdf_path)

In [9]:
def write_condition_reports(subject, experiment, condition, probe):
    drift_tracking_chans = ea.get_channels(subject, experiment, probe, "drift_tracking")
    csd_params_path = ea.get_datapath("sharp_wave_detection_params.json", subject, experiment)
    with open(csd_params_path) as csd_params_file:
        csd_params = json.load(csd_params_file)

    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="lf.bin")
    pdf_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, ext="CSD_SR_markers.pdf")

    for bin_path, pdf_path in zip(bin_paths, pdf_paths):
        write_file_reports(bin_path, pdf_path, drift_tracking_chans, csd_params)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [20]:
write_condition_reports("Allan", "ketamine", "all", "imec0")

nChan: 385, nFileSamp: 17610153
14:50:32: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/4-20-2021/4.20.2021_g0/4.20.2021_g0_imec0/4.20.2021_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 17999996
15:01:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/4-20-2021/4.20.2021_g1/4.20.2021_g1_imec0/4.20.2021_g1_t0.imec0.lf.bin
nChan: 385, nFileSamp: 17999996
15:13:26: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/4-20-2021/4.20.2021_g1/4.20.2021_g1_imec0/4.20.2021_g1_t1.imec0.lf.bin
nChan: 385, nFileSamp: 8530138
15:18:15: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/4-20-2021/4.20.2021_g1/4.20.2021_g1_imec0/4.20.2021_g1_t2.imec0.lf.bin
